# PLIP — Protein ligand interaction profiling

See [the tutorial](/Tutorials/protein-ligand-interactions.ipynb).

In [ ]:
# |hide
WORK_DIR = '~/qdx/plip/'
!rm -r $WORK_DIR || true
!mkdir -p $WORK_DIR
%cd -q $WORK_DIR

In [ ]:
# Get a pdb to work with - we use the pdb-tools cli here but you can download directly from rcsb.org
!pdb_fetch 1MBN > 1MBN.pdb

In [ ]:
from pathlib import Path
import rush

client =  rush.build_blocking_provider_with_functions(batch_tags=["plip"])
output_conformer, output_txt = client.plip(Path.cwd() / "1MBN.pdb")

print(output_conformer.download().open().read()[0:50], "...")
print(output_txt.download().open().read()[0:50], "...")

2024-03-16 10:17:05,990 - rush - INFO - Argument ba36d984-4668-4b3e-9a4c-58a173f87ff4 is now ModuleInstanceStatus.RESOLVING
2024-03-16 10:17:12,494 - rush - INFO - Argument ba36d984-4668-4b3e-9a4c-58a173f87ff4 is now ModuleInstanceStatus.ADMITTED
2024-03-16 10:17:17,876 - rush - INFO - Argument ba36d984-4668-4b3e-9a4c-58a173f87ff4 is now ModuleInstanceStatus.DISPATCHED
2024-03-16 10:17:18,951 - rush - INFO - Argument ba36d984-4668-4b3e-9a4c-58a173f87ff4 is now ModuleInstanceStatus.AWAITING_UPLOAD
{"topology": {"version": null, "symbols": ["N", "C ...
Prediction of noncovalent interactions for PDB str ...
